In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
train = pd.read_csv('../input/bigmart-sales-data/Train.csv')
test = pd.read_csv('../input/bigmart-sales-data/Test.csv')

In [ ]:
train

In [ ]:
test

concate both perform feature engineering and then divide them later again.

In [ ]:
train['source'] = 'train'
test['source'] = 'test'
test['Item_Outlet_Sales'] = 0
data = pd.concat([train,test],sort=False)

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data['Item_Outlet_Sales'].describe()

as Item_Outlet_Sales is the main prediction factor lets see how i

In [ ]:
import seaborn as sns
sns.distplot(data['Item_Outlet_Sales'])
plt.show()

Deviate from the normal distribution and 
Having appreciable positive skewness.

In [ ]:
data.info()

we are with both object type and numerical lets saparate thous 

In [ ]:
cat_var = data.select_dtypes(include =[np.object])
cat_var.shape

In [ ]:
num_var = data.select_dtypes(include=[np.float64])
num_var.shape

In [ ]:
cat_var.isnull().sum()

in categorical var we have 4016 missing values of Outlet_size and lets check in numerical var

In [ ]:
num_var.isnull().sum()

we have 2439 missing values in Item_Weight 

lets see Unique values of each attribute 

In [ ]:
data.apply(lambda x : len(x.unique()))

by this we can say that there are totally 1559 products and 10 Outlets and there are totally 16 types of Item 

In [ ]:
data.boxplot(column='Item_Outlet_Sales',by='Item_Fat_Content')

by this we can say that Some of ‘Low Fat’ values mis-coded as ‘low fat’ and ‘LF’. Also, some of ‘Regular’ are mentioned as ‘regular’.

In [ ]:
for var in cat_var:
    data[var].fillna(method='ffill',inplace=True)

In [ ]:
for var in num_var:
    mean = np.around(np.mean(data[var]))
    data[var].fillna(mean,inplace = True)

In [ ]:
data.isnull().sum()

we placed all the missing values with mean so by that there is no missing values in data now 

In [ ]:
plt.figure(figsize =(10,9))
plt.subplot(311)

ax = sns.boxplot(x = 'Outlet_Size',y = 'Item_Outlet_Sales',data=data ,palette="Set1")
ax.set_title("Outlet_size vs Item_Outlet_Sales")
plt.subplots_adjust(wspace = 0.2, hspace = 0.4,top = 1.5)

In [ ]:
plt.figure(figsize=(10,9))
plt.subplot(212)

ax = sns.boxplot(x ='Outlet_Type' ,y='Item_Outlet_Sales',data=data,palette="Set1" )

and in our data set one attribute Item_visibility which mean The % of total display area of all products in a store allocated to the particular product but does not make sense with 0 so lets consider it as missing values and impute wirh mean visibility

In [ ]:
#Determine average visibility of a product
visibility_avg = data.pivot_table(values='Item_Visibility', index='Item_Identifier')

#Impute 0 values with mean visibility of that product:
missing_values = (data['Item_Visibility'] == 0)

data.loc[missing_values,'Item_Visibility'] = data.loc[missing_values,'Item_Identifier'].apply(lambda x: visibility_avg.at[x, 'Item_Visibility'])


if we observer in Item_Identifier there unique key have 'FD' if product is related to food and 'DR' if the product is Drink 
and 'NC' if it is Non_consumable so lets make a new variable by combining them 

In [ ]:
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x:x[0:2])
data['Item_Type_Combined'] = data['Item_Type_Combined'].map({'FD':'food','NC':'Non_consum','DR':'Drink'})
data['Item_Type_Combined'].value_counts()

In [ ]:
data.info()

In [ ]:
plt.figure(figsize=(10,9))
plt.subplot(212)

ax = sns.boxplot(x ='Item_Type_Combined' ,y='Item_Outlet_Sales',data=data,palette="Set1" )

In [ ]:
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF':'Low Fat','reg':'Regular','low fat':'Low Fat'})

In [ ]:
data['Item_Fat_Content'].value_counts()

In [ ]:
plt.figure(figsize=(10,9))
plt.subplot(212)

ax = sns.boxplot(x ='Item_Fat_Content' ,y='Item_Outlet_Sales',data=data,palette="Set1" )

In [ ]:
data.groupby('Outlet_Establishment_Year')['Item_Outlet_Sales'].mean().plot.bar()

it will make more sense if we have noe many years did the Outlet is being woring for so lets use Outlet_Establishment_Year and make a new attribute Outlet_years this can be done by 

In [ ]:
data['Outlet_Establishment_Year'].describe()

In [ ]:
data['Outlet_Year'] = 2019 - data['Outlet_Establishment_Year']
data['Outlet_Year'].describe()

as sklearn only accepts only numerical variables, we  converted all categories into numeric types 

In [ ]:
from sklearn.preprocessing import LabelEncoder

encode = LabelEncoder()

data['Outlet'] = encode.fit_transform(data['Outlet_Identifier'])

var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
for i in var_mod:
    data[i] = encode.fit_transform(data[i])

In [ ]:
data.dtypes

In [ ]:
data = pd.get_dummies(data,columns = var_mod)

In [ ]:
data.dtypes

In [ ]:
plt.figure(figsize=(10,9))
plt.subplot(212)

ax = sns.boxplot(x ='Outlet_Year' ,y='Item_Outlet_Sales',data=data,palette="Set1" )

In [ ]:
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]

In [ ]:
train.shape , test.shape

Now our data is ready for model building

conclusion ---------

1 . Stores located in urban or Tier 1 cities should have higher sales because of the higher income levels of people there.
2 . Stores with higher population will have more sales 
3 . Stores located within popular marketplaces should have higher sales because of better access to customers.
4 . Stores with large space . Stores which are very big in size should have higher sales as they act like one-stop-shops and people would prefer getting everything from one place

according to product :

1 . Brand,packaging,Advertising,Visibility in Store,offers will show very good affect on product sales 